In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv('D:/datasets/train.csv')
data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [3]:
data.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

In [4]:
data.shape

(891, 8)

In [5]:
data.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [6]:
from sklearn.model_selection import train_test_split
xtrain, xtest,ytrain,ytest = train_test_split(data.drop('Survived', axis=1), data['Survived'], test_size=0.1)
xtrain.shape

(801, 7)

In [7]:
data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [9]:
#age column data missng in percentage
age_datamissing=data['Age'].isnull().sum()/(data.shape[0] / 100)
age_datamissing

19.865319865319865

In [10]:
embarked_datamissing = data['Embarked'].isnull().sum()/(data.shape[0]/100)
embarked_datamissing

0.2244668911335578

In [11]:
#handling the missing values of Age and Embarked columns
si_age = SimpleImputer(strategy='mean')
si_embarked = SimpleImputer(strategy='most_frequent')

train_age_transformed=si_age.fit_transform(xtrain[['Age']])
train_embarked_transformed=si_embarked.fit_transform(xtrain[['Embarked']])

test_age_transformed=si_age.transform(xtest[['Age']])
test_embarked_transformed=si_embarked.transform(xtest[['Embarked']])

In [12]:
test_age_transformed[5:11]

array([[17.        ],
       [27.        ],
       [20.        ],
       [29.74097523],
       [29.74097523],
       [29.74097523]])

In [13]:
data['Sex'].nunique()

2

ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')<br>ohe_embarked = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore') <br>
NOTE: while using OneHotEncoder, we can not use both drop='first' and handle_unknown='ignore'. Example if we had three features
a,b,c if we encode all the features it would be [1 0 0 ], [0 1 0] anf [0 0 1] respectively. drop='False' would drop the column
a which means a will be representated by [0 0] later on. Similar to drop='False', handle_unknown='ignore' would also make the
new feature's vector to all zeros. That means feature a's vector values and newly introduced feature's vector value will be all
zeros which is definitely create a confusion... That's why sklean doesnot allow both the parameters to be used at a single time.
<br> <br>
The reasons why we should not use drop='first' <br>
https://www.youtube.com/watch?v=NYtwyvyvDEk <br>
https://inmachineswetrust.com/posts/drop-first-columns/

In [14]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False, handle_unknown='ignore')


ohe.fit(xtrain[['Sex']])
ohe_embarked.fit(train_embarked_transformed)

train_sex = ohe.transform(xtrain[['Sex']])
test_sex =  ohe.transform(xtest[['Sex']])

train_embarked = ohe_embarked.transform(train_embarked_transformed)
test_embarked = ohe_embarked.transform(test_embarked_transformed)

train_sex[0:4]

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [15]:
train_embarked[21:27]

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

# using pipeline to make all the work efficient.

In [51]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [52]:
xtrain.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
686,3,male,14.0,4,1,39.6875,S
471,3,male,38.0,0,0,8.6625,S
100,3,female,28.0,0,0,7.8958,S


Below, I have used 2 ColumnTrasformers. 1st one to handle the missing values og Age and Embarked column and
2nd to convert the nominal categorical features into numerical. After filling the missing value in the embarked column, as it 
is categorical column, I converted it into numerical in 2nd columntransformer. <br>
While implementing 2nd columntransformer, I gave index value in order to track the column as 1st ColumnTransformer would return
numpy array. I did this for an experiement and still in search for the optimum way to do this.

In [85]:
ctf1 = ColumnTransformer(transformers=[
    ('impute_age', SimpleImputer(strategy='mean'),['Age']),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),['Embarked'])
], remainder='passthrough')

In [86]:
# a = ctf1.fit_transform(xtrain)
# a[0]

In [87]:
ctf2 = ColumnTransformer(transformers=[
    ('ohe_Sex', OneHotEncoder(handle_unknown='ignore', sparse=False),[1,3])
],remainder='passthrough')

# ctf2.fit_transform(a)[0]

In [88]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [89]:
pipe= Pipeline(
[
    ('1',ctf1),
    ('2',ctf2),
    ('3',lr)
]
)

In [90]:
pipe.fit(xtrain,ytrain)
pred = pipe.predict(xtest)

C:\Users\dell\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [91]:
from sklearn.metrics import accuracy_score

In [92]:
accuracy= accuracy_score(ytest, pred)
accuracy

0.7666666666666667

# using cross_val_score inplace of train_test_split

In [102]:
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

In [93]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [96]:
X = data.drop(['Survived'], axis=1)
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [97]:
y = data['Survived']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [98]:
ct1 = ColumnTransformer(transformers=[
    ('impute_age', SimpleImputer(strategy='mean'),['Age']),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),['Embarked'])
], remainder='passthrough')

ct2 = ColumnTransformer(transformers=[
    ('ohe_Sex', OneHotEncoder(handle_unknown='ignore', sparse=False),[1,3])
],remainder='passthrough')

lrm = LogisticRegression()

In [99]:
pipe= Pipeline(
[
    ('1',ct1),
    ('2',ct2),
    ('3',lrm)
]
)

In [101]:
y.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [108]:
fold = KFold(10, shuffle=True, random_state=1)
cross = cross_val_score(pipe,X,y, cv=fold)
cross

C:\Users\dell\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dell\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

array([0.77777778, 0.80898876, 0.75280899, 0.75280899, 0.82022472,
       0.79775281, 0.87640449, 0.80898876, 0.80898876, 0.78651685])

In [109]:
cross.mean()

0.7991260923845193

Using cross_val_score what we can say is, with the samedata set and the model we got minimum of75% accuracy and 80% of max 
accuracy and the average is 0.79%